## Similarity Analysis

In [2]:
from similarity import SimilarityPredictions
from environment_configuration import *

# Import packages
import pickle
import pandas as pd
from pathlib import Path
import gc

Script: 01.01.02 [Import Packages] completed
Script: 01.01.03 [Update Data Paths] completed
Script: 01.01.04 [Define Other Global Variables] completed


In [3]:
#Load clean producs data
products_clean = pd.read_pickle("../data/products_clean_idx.pkl")

In [4]:
#load collaborative product embeddings
prod_embeddings = pd.read_pickle("../data/product_embeddings_ids_only.pkl")

In [5]:
def lookup_asin_by_title(prod_title):
    return products_clean[products_clean.title.str.contains(prod_title, case=False)]

In [6]:
lookup_asin_by_title('xbox')

,asin,description,title,category2_t,category3_t,category4_t,category5_t,category6_t,hasDescription,price_t,containsAnySalesRank,numberQuestions,numberReviews,meanStarRating,star1Rating,star2Rating,star3Rating,star4Rating,star5Rating,electronicsRankBin
idx,,,,,,,,,,,,,,,,,,,,
54610,B00062NHTS,"DVD players, VCRs, DBS receivers, video game s...",Nyrius SW200 S-Video Composite &amp; Digital O...,Accessories & Supplies,Audio & Video Accessories,Distribution,Selector Switch Boxes,Unknown,1,29.97,0,0,15.0,4.266667,0.066667,0.066667,0.000000,0.266667,0.600000,Unknown
14464,B0006PY3DY,Developed in conjunction with the Game Audio N...,Xbox Spherex 51000 6-Piece 5.1 Surround Sound ...,Computers & Accessories,Cables & Accessories,Computer Speakers,Unknown,Unknown,1,24.72,0,5,6.0,4.333333,0.000000,0.000000,0.166667,0.333333,0.500000,Unknown
15306,B000812QC6,Get more hookup flexibility for your digital c...,Nyrius SW100 Digital Audio Optical Toslink 3 I...,Accessories & Supplies,Audio & Video Accessories,Distribution,Selector Switch Boxes,Unknown,1,19.97,1,10,129.0,3.775194,0.186047,0.046512,0.062016,0.217054,0.488372,Unknown
31864,B000B525DY,If you have more component video sources than ...,Nyrius SW201 HD Component Video YPbPr &amp; Di...,Accessories & Supplies,Audio & Video Accessories,Distribution,Selector Switch Boxes,Unknown,1,49.99,0,10,30.0,4.500000,0.033333,0.033333,0.066667,0.133333,0.733333,Unknown
16016,B000CCXCYC,Make your Xbox 360&#xAE; part of your home ent...,Logitech Harmony Xbox 360 Remote (Discontinued...,Accessories & Supplies,Audio & Video Accessories,Remote Controls,TV Remote Controls,Unknown,1,139.99,0,9,67.0,4.268657,0.029851,0.044776,0.104478,0.268657,0.552239,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42301,B00FX9WJNO,6ft High Speed HDMI Cable with Ethernet 28 AWG...,HDMI 2 Pack - PREMIUM SERIES GOLD Plated 6 ft ...,Accessories & Supplies,Audio & Video Accessories,Cables & Interconnects,Video Cables,HDMI Cables,1,35.00,0,8,20.0,4.850000,0.000000,0.000000,0.050000,0.050000,0.900000,Unknown
43870,B00G059G86,The multi-purpose quiet fan can be placed on t...,AC Infinity AI-MPF80A Quiet 80mm USB Fan for R...,Computers & Accessories,Computer Components,Fans & Cooling,Case Fans,Unknown,1,9.99,1,0,10.0,4.800000,0.000000,0.000000,0.000000,0.200000,0.800000,Unknown
42743,B00G05A2MU,The multi-purpose quiet fan can be placed on t...,AC Infinity AI-MPF120A Quiet 120mm USB Fan for...,Computers & Accessories,Computer Components,Fans & Cooling,Case Fans,Unknown,1,13.99,1,10,20.0,4.800000,0.000000,0.000000,0.050000,0.100000,0.850000,Unknown


### Sklearn similarity compare gives memory error

In [48]:
def get_detailed_recs(prodIdx, embeddings, comp_type="cosine", file_path=''):
    #convert embeddings to compressed format
    
    #get similar movies
    sim_model = SimilarityPredictions(embeddings, similarity_metric=comp_type)
    output = sim_model.predict_similar_items(seed_item=prodIdx, n=20)
    similar_prods = pd.DataFrame(output)
    similar_prods.set_index('item_id', inplace=True)
    sim_df = pd.merge(products_clean, similar_prods, left_index=True, right_index=True)
    sim_df.sort_values('similarity_score', ascending=False, inplace=True)
    
    #save recs locally
    if(len(file_path)>0):
        sim_df.head(20).to_csv(file_path, index=False, header=True)
        
    return sim_df.head(20)

In [93]:
gc.collect()

13419

In [54]:
#search for Seiki SE50UY04 50-Inch 4K UHD 120Hz LED HDTV
# B00ES5Q6E2 => idx 2411
get_detailed_recs(50295,prod_embeddings[:10000], comp_type="euclidean")

KeyError: 50295

In [94]:
import numpy as np
np.dot(prod_embeddings[:10000].values, prod_embeddings[:10000].values.T)

MemoryError: 

In [7]:
prod_embeddings.shape

(62990, 100)

### Try scipy distance calculation

In [8]:
import scipy.spatial.distance
def cos_cdist(matrix, vector):
    """
    Compute the cosine distances between each row of matrix and vector.
    """
    v = vector.reshape(1, -1)
    return 1-scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

In [9]:
x = prod_embeddings.iloc[54610]
y = cos_cdist(prod_embeddings.values, x.values)

In [10]:
len(y)

62990

In [11]:
sim_results = pd.DataFrame(y, columns=['cdist'])

In [12]:
sim_results.nlargest(10, 'cdist').round(6)

,cdist
54610,1.000000
50801,0.747855
55528,0.745170
3659,0.744890
3611,0.743781
43329,0.741314
12293,0.731471
37041,0.724865
4937,0.723555
10258,0.723492


In [15]:
products_clean.loc[3659]

asin                                                           B0055QYJIS
description             Charge your HP TouchPad without wires using th...
title                            HP Touchstone Charging Dock for TouchPad
category2_t                                       Computers & Accessories
category3_t                               Touch Screen Tablet Accessories
category4_t                                           Chargers & Adapters
category5_t                                                       Unknown
category6_t                                                       Unknown
hasDescription                                                          1
price_t                                                             39.99
containsAnySalesRank                                                    0
numberQuestions                                                        10
numberReviews                                                         155
meanStarRating                        